In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import read_structure, molecule
from matplotlib import pyplot as plt
from pennylane import commutator

In [2]:
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

def build_cost_h(n):
    coeffs = [-1.0] * n
    ops = [qml.PauliZ(i) for i in range(n)]
    return qml.Hamiltonian(coeffs, ops)

def build_driver_h(n):
    coeffs = [-1.0] * n
    ops = [qml.PauliX(i) for i in range(n)]
    return qml.Hamiltonian(coeffs, ops)

cost_h = build_cost_h(n_qubits)
print(cost_h)

driver_h = build_driver_h(n_qubits)
print(driver_h)


-1.0 * Z(0) + -1.0 * Z(1) + -1.0 * Z(2) + -1.0 * Z(3)
-1.0 * X(0) + -1.0 * X(1) + -1.0 * X(2) + -1.0 * X(3)


In [4]:
def build_commutator_hamiltonian(H_d, H_c):
    comm_terms = []
    coeffs_d, ops_d = H_d.terms()
    coeffs_c, ops_c = H_c.terms()
    for c1, op1 in zip(coeffs_d, ops_d):
        for c2, op2 in zip(coeffs_c, ops_c):
            try:
                comm = commutator(op1, op2)
                if isinstance(comm, qml.ops.op_math.Sum) and len(comm.operands) == 0:
                    continue
                comm_terms.append(1j * c1 * c2 * comm)
            except Exception as e:
                print(f"跳过 [{op1}, {op2}]：{e}")
                continue
    return sum(comm_terms).simplify()

comm_h = build_commutator_hamiltonian(driver_h, cost_h)
comm_hd= build_commutator_hamiltonian(driver_h,driver_h)
print(comm_h)
print(comm_hd)

(2+0j) * Y(0) + (2+0j) * Y(1) + (2+0j) * Y(2) + (2+0j) * Y(3)
0 * I()
